In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from pyzotero import zotero

load_dotenv()

ZOTERO_USER_ID = os.getenv('ZOTERO_USER_ID')
ZOTERO_API_KEY = os.getenv('ZOTERO_API_KEY')

zot = zotero.Zotero(ZOTERO_USER_ID, 'user', ZOTERO_API_KEY)

In [19]:
import json

library_items = zot.everything(zot.collection_items_top('SVUM4G2M', limit=106))
library = pd.read_csv('zotero_pdf_matches.csv', encoding='windows-1252')

for item in library_items:
    title = item['data']['title'].replace('’', "'").replace('–', '-')
    matched_rows = library[library['title'] == title]
    if not matched_rows.empty:
        pdf_name = matched_rows.iloc[0]['pdf_name']
        with open(f"zotero_cache_metadata/{pdf_name.replace('.pdf', '')}.json", "w") as f:
            json.dump(item, f, indent=4)
    else:
        print(f"Title: {title} -> PDF Name: Not Found")

In [ ]:
import fitz

doc = fitz.open('/home/mlevi/OneDrive/Zotero/Afgan et al. - 2025 - Faceexpr Personalized Facial Expression Generation Via Attention-Focused U-Net Feature Fusion in Di.pdf', filetype="pdf")
full_text = ""
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    text = page.get_text()
    full_text += text
doc.close()

' \nContents lists available at ScienceDirect\nInformation Fusion\njournal homepage: www.elsevier.com/locate/inffus\n \nFull length article\nFaceExpr: Personalized facial expression generation via attention-focused \nU-Net feature fusion in diffusion models\nMuhammad Sher Afgan a\n, Bin Liu a\n,∗, Mamoona Naveed Asghar b, Wajahat Khalid a\n, \nKai Zou a, Dianmo Sheng a\na School of Cyber Science and Technology, University of Science and Technology of China, 96 Jinzhai Road, Hefei, 230026, Anhui, China\nb School of Computer Science, University of Galway, University Road, Galway, H91 TK33, Galway, Ireland\nA R T I C L E  I N F O\nDataset link: https://github.com/MSAfganUST\nC/FaceExpr.git\nKeywords:\nAttention\nDiffusion\nExpressions synthesis\nFusion\nIdentity preservation\nText-to-image\n \nA B S T R A C T\nText-to-image diffusion models have revolutionized image generation by creating high-quality visuals from text \ndescriptions. Despite their potential for personalized text-to-image

In [ ]:
from docling.document_converter import DocumentConverter

from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import ThreadedPdfPipelineOptions
from docling.datamodel.base_models import InputFormat
from docling.datamodel.settings import settings

# --- Your main settings to add ---

# 1. Set the global page batch size to a small number
# The default is 4, try setting it to 1
settings.perf.page_batch_size = 1

# 2. Configure pipeline-specific batch sizes
# We are manually setting them to 1 to force sequential processing
pipeline_options = ThreadedPdfPipelineOptions(
    ocr_batch_size=1,     # Default is 4
    layout_batch_size=1,  # Default is 4
    table_batch_size=1    # Default is 4
)

# 3. Pass these options when you create the DocumentConverter
doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

# --- Now, process your large file ---
source = '/home/mlevi/OneDrive/Zotero/Afgan et al. - 2025 - Faceexpr Personalized Facial Expression Generation Via Attention-Focused U-Net Feature Fusion in Di.pdf'
doc = doc_converter.convert(source).document

print(doc.export_to_markdown())

/home/mlevi/Work/research-assistant/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-31 23:55:26,246 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-31 23:55:26,331 - INFO - Going to convert document batch...
2025-10-31 23:55:26,332 - INFO - Initializing pipeline for StandardPdfPipeline with options hash beef5fd6685ff8fca4714abbf6d083c5
2025-10-31 23:55:26,338 - INFO - Loading plugin 'docling_defaults'
2025-10-31 23:55:26,341 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-10-31 23:55:26,349 - INFO - Loading plugin 'docling_defaults'
2025-10-31 23:55:26,352 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-10-31 23:55:26,353 - INFO - rapidocr cannot be used because onnxruntime is not installed.
2025-10-31 23

In [ ]:
from difflib import SequenceMatcher

def get_similarity(str1, str2):
    return SequenceMatcher(None, str1.lower(), str2.lower()).ratio()

def find_best_match(title, pdf_list, threshold=0.5):
    best_match = None
    best_score = 0

    for pdf in pdf_list:
        pdf_name = pdf.replace('.pdf', '').replace('et al.', '')
        similarity = get_similarity(title, pdf_name)

        if similarity > best_score:
            best_score = similarity
            best_match = pdf

    if best_score >= threshold:
        return best_match, best_score
    else:
        return None, best_score

In [ ]:
matched_count = 0
unmatched_count = 0
low_confidence_matches = []
titles = []
pdf_names = []

for metadata in zot.everything(zot.collection_items_top('SVUM4G2M', limit=106)):
    if 'data' in metadata and 'title' in metadata['data']:
        title = metadata['data']['title']

        best_match, similarity = find_best_match(title, pdf_list, threshold=0.5)

        if best_match:
            matched_count += 1
            print(f"✓ Match (score: {similarity:.2f}): {title[:60]}... → {best_match}")
            titles.append(title)
            pdf_names.append(best_match)

            if similarity < 0.7:
                low_confidence_matches.append((title, best_match, similarity))
        else:
            unmatched_count += 1
            print(f"✗ No match (best score: {similarity:.2f}): {title[:60]}...")
            titles.append(title)
            pdf_names.append('')

print(f"\n{'='*80}")
print(f"Summary: {matched_count} matched, {unmatched_count} unmatched")
print(f"Low confidence matches (< 0.7): {len(low_confidence_matches)}")

if low_confidence_matches:
    print(f"\n{'='*80}")
    print("Low confidence matches to review:")
    for title, pdf, score in low_confidence_matches[:10]:
        print(f"  Score {score:.2f}: {title[:50]}... → {pdf}")

✓ Match (score: 0.67): Facial expression synthesis based on denoising diffusion pro... → Ho et al. - 2020 - Denoising Diffusion Probabilistic Models.pdf
✓ Match (score: 1.00): Uncertainty-Aware Semi-Supervised Learning of 3D Face Riggin... → Uncertainty-Aware Semi-Supervised Learning of 3D Face Rigging from Single Image.pdf
✓ Match (score: 1.00): Talking Face Generation with Expression-Tailored Generative ... → Talking Face Generation with Expression-Tailored Generative Adversarial Network.pdf
✓ Match (score: 0.99): Synthesis of Facial Expressions in Photographs: Characterist... → Synthesis of Facial Expressions in Photographs Characteristics, Approaches, and Challenges.pdf
✓ Match (score: 0.88): Reviving Intentional Facial Expressions: an Interface for AL... → Reviving Intentional Facial Expressions an Interface for ALS Patients using Brain Decoding and Imag.pdf
✓ Match (score: 1.00): Fine-grained Micro-Expression Generation based on Thin-Plate... → Fine-grained Micro-Expression Gener

In [ ]:
pd.DataFrame({'title': titles, 'pdf_name': pdf_names}).to_csv('zotero_pdf_matches.csv', index=False)

In [ ]:
library = pd.read_csv('zotero_pdf_matches.csv', encoding='windows-1252')
library.head()

0
1
2
3
4


In [20]:
from os import getenv
from agno.agent import Agent
from agno.models.openrouter import OpenRouter

agent = Agent(
    model=OpenRouter(id="gpt-5-mini", api_key="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"),
    markdown=True
)

# Print the response in the terminal
agent.print_response("Share a 2 sentence horror story.")

2025-10-31 15:42:52,576 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-31 15:43:05,657 - INFO - HTTP Request: POST https://os-api.agno.com/telemetry/runs "HTTP/1.1 201 Created"


┏━ Message ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                              ┃
┃ Share a 2 sentence horror story.                                             ┃
┃                                                                              ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Response (13.8s) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                              ┃
┃ I woke to my phone lighting up with a photo of me sleeping, taken from       ┃
┃ inside my dark bedroom. I live alone.                                        ┃
┃                                                                              ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
